In [121]:
from typing import Dict, Any, List, Optional
import pandas as pd
from datetime import datetime, timedelta
from prometheus_api_client.utils import parse_datetime
import requests
from prometheus_api_client import PrometheusConnect

In [122]:
prom = PrometheusConnect(
            url="http://localhost:8082",
        )


In [123]:
prom.all_metrics()

['ALERTS',
 'ALERTS_FOR_STATE',
 'aggregator_discovery_aggregation_count_total',
 'aggregator_unavailable_apiservice',
 'apiextensions_apiserver_validation_ratcheting_seconds_bucket',
 'apiextensions_apiserver_validation_ratcheting_seconds_count',
 'apiextensions_apiserver_validation_ratcheting_seconds_sum',
 'apiextensions_openapi_v2_regeneration_count_total',
 'apiextensions_openapi_v3_regeneration_count_total',
 'apiserver_admission_controller_admission_duration_seconds_bucket',
 'apiserver_admission_controller_admission_duration_seconds_count',
 'apiserver_admission_controller_admission_duration_seconds_sum',
 'apiserver_admission_step_admission_duration_seconds_bucket',
 'apiserver_admission_step_admission_duration_seconds_count',
 'apiserver_admission_step_admission_duration_seconds_sum',
 'apiserver_admission_step_admission_duration_seconds_summary',
 'apiserver_admission_step_admission_duration_seconds_summary_count',
 'apiserver_admission_step_admission_duration_seconds_summar

In [267]:
PROMQL = {
        # Cost forecasting - cluster-level cost trends for OpenCost overlay in Grafana
    "cost_usd": (
        "sum(node_total_hourly_cost) by (clusterName)"
    ),
    
    # CPU utilization forecasting - cluster-level CPU trends for capacity planning
    "cpu_pct": (
        "100 * (1 - avg by (clusterName) "
        "(rate(node_cpu_seconds_total{mode=\"idle\"}[5m])))"
    ),
    
    # Memory utilization forecasting - cluster-level memory trends for capacity planning  
    "mem_pct": (
        "100 * (1 - avg by (clusterName) "
        "(node_memory_MemAvailable_bytes / node_memory_MemTotal_bytes))"
    ),
    
    # ---------- core per-node metrics ----------
    "cost_usd_per_node": """
        sum by (clusterName, node) (node_total_hourly_cost)
    """,

    # Per-node CPU utilization for node-level capacity forecasting and visualization
    "cpu_pct_per_node": (
        "100 * (1 - avg by (clusterName, nodename) "
        "(rate(node_cpu_seconds_total{mode=\"idle\"}[5m])))"
    ),
    
    # Per-node memory utilization for node-level capacity forecasting and visualization
   "mem_pct_per_node": (
        "100 * (1 - avg by (clusterName, nodename) "
        "(node_memory_MemAvailable_bytes / node_memory_MemTotal_bytes))"
    ),

    # Node count for scaling-trend forecasting
    # Counts only Ready nodes per cluster (requires kube-state-metrics)
    "node_count": """
        count by (clusterName) (
          kube_node_status_condition{condition="Ready",status="true"}
        )
    """,
    # CPU capacity forecasting - total CPU cores per cluster (for growth planning)
    "cpu_total_cores": (
        "sum by (clusterName, instance) (machine_cpu_cores)"
    ),
     "node_count_per_cluster": "count by (clusterName) (kube_node_status_condition{condition=\"Ready\",status=\"true\"})",
}

In [312]:

endTime=datetime.now()
result = prom.custom_query_range(
                    query=PROMQL["mem_pct_per_node"],
                    end_time=endTime,
                    start_time=endTime - timedelta(days=5)-timedelta(minutes=5),
                    step="1h"
                )

In [313]:
result



[{'metric': {'clusterName': 'aws-ramesses-regional-0',
   'nodename': 'aws-ramesses-regional-0-cp-0'},
  'values': [[1752004884, '30.749283390713423'],
   [1752008484, '30.94092575634638'],
   [1752012084, '31.116966436003523'],
   [1752015684, '31.448725854030823'],
   [1752019284, '29.321182568047444'],
   [1752022884, '30.072050970807453'],
   [1752026484, '30.43317534555995'],
   [1752030084, '30.96591826604651'],
   [1752033684, '31.09311672495768'],
   [1752037284, '31.413547530238205'],
   [1752040884, '29.405352173280075'],
   [1752044484, '30.036524838728752'],
   [1752048084, '30.73368170473759'],
   [1752051684, '31.075179754775284'],
   [1752055284, '31.442465304881296'],
   [1752058884, '31.571452492120898'],
   [1752062484, '29.422196031706203'],
   [1752066084, '29.94927464581937'],
   [1752069684, '30.72503618448348'],
   [1752073284, '30.849998732984098'],
   [1752076884, '31.55068336875181'],
   [1752080484, '31.538957260820933'],
   [1752084084, '29.387663637587735']